# Workgroup 3

**Group 3**: Valerie Dube, Erzo Garay, Juan Marcos Guerrero y Matías Villalba,

## 1. Neyman Orthogonality Proof

## 2. Code Section

### 2.1. Orthogonal Learning

### 2.2. Double Lasso - Using School data

In [ ]:
# val